In [18]:
# rsa simple implementing
# 
# p, q : two different primes
# n = p * q  : modulus
# l = lcm(p - 1, q - 1)
# e: 1 < e < l and gdc(e, l) == 1
#    (e, n) : public key
# d: 1 < d < l and d * e mod l == 1
#    (d, n) : private key
# 
# m: message, m < n
# encrypted = m ** e mod n
# decrypted = encrypted ** d mod n
#
#

In [18]:
# base utility functions to generate prime
def primes_generator():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last

# base util function
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x

# base util function
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [19]:
def get_n(p, q):
    return p * q


def get_l(p, q):
    return lcm(p - 1, q - 1)


def get_e(l):
    for e in range(2, l):
        if gcd(e, l) == 1:
            return e
        else:
            continue
            
        return 0

    
def get_d(l, e):
    for d in range(2, l):
        if e * d % l == 1:
            return d
        else:
            continue
            
        return 0

In [20]:
def calc_rsa_elements(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['l'] = get_l(p, q)
    l = rsa['l']
    e = get_e(l)
    if e == 0:
        rsa['ready'] = False
        return rsa
    
    d = get_d(l, e)
    if d == 0:
        rsa['ready'] = False
        return rsa
    
    rsa['e'] = e
    rsa['d'] = d
    rsa['ready'] = True
    
    return rsa

In [21]:
def try_to_gen_rsa():
    primes = primes_generator()
    for i in (1, 3):
        next(primes)
    p = next(primes)
    for i in range(1, 1000):
        q = next(primes)
        rsa = calc_rsa_elements(p, q)
        if not rsa['ready']:
            continue
        return rsa

In [41]:
def verify_rsa(rsa, msgs):
    for m in msgs:
        encrypt = m ** rsa['e'] % rsa['n']
        decrypt = encrypt ** rsa['d'] % rsa['n']
        print("m: {}, encrypted: {}, decrypted: {}, (m == decrypted): {}".format(m, encrypt, decrypt, m == decrypt))

In [42]:
def main():
    rsa = try_to_gen_rsa()
    print('rsa -> {}'.format(rsa))
    msgs = range(1, 20)
    verify_rsa(rsa, msgs)

In [43]:
main()

rsa -> {'p': 5, 'q': 7, 'n': 35, 'l': 12, 'e': 5, 'd': 5, 'ready': True}
m: 1, encrypted: 1, decrypted: 1, (m == decrypted): True
m: 2, encrypted: 32, decrypted: 2, (m == decrypted): True
m: 3, encrypted: 33, decrypted: 3, (m == decrypted): True
m: 4, encrypted: 9, decrypted: 4, (m == decrypted): True
m: 5, encrypted: 10, decrypted: 5, (m == decrypted): True
m: 6, encrypted: 6, decrypted: 6, (m == decrypted): True
m: 7, encrypted: 7, decrypted: 7, (m == decrypted): True
m: 8, encrypted: 8, decrypted: 8, (m == decrypted): True
m: 9, encrypted: 4, decrypted: 9, (m == decrypted): True
m: 10, encrypted: 5, decrypted: 10, (m == decrypted): True
m: 11, encrypted: 16, decrypted: 11, (m == decrypted): True
m: 12, encrypted: 17, decrypted: 12, (m == decrypted): True
m: 13, encrypted: 13, decrypted: 13, (m == decrypted): True
m: 14, encrypted: 14, decrypted: 14, (m == decrypted): True
m: 15, encrypted: 15, decrypted: 15, (m == decrypted): True
m: 16, encrypted: 11, decrypted: 16, (m == decrypted